# Recommendation - Model 🍿

---

<img src="https://visithrastnik.si/uploads/tic/public/generic_list_item/6-kulturna_prireditev_v_avli_kulturnega_centra_zagorje_ob_savi.jpg" />

---

Now, time for the exciting part! We will train a Machine Learning model based on our previous **ratings** sparse matrix, so that it creates a recommendation engine automatically! 

First, load again the dataframe `movies` and `ratings`

In [1]:
### TODO: load the movies and ratings datasets
import pandas as pd
movies = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")

print(movies.head())
print(ratings.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


**Q1**. Start by loading all the pickle you saved during last challenge: `ratings_matrix`, `idx_to_mid`, `mid_to_idx`, `uid_to_idx`, `idx_to_uid`

In [2]:
import pickle

ratings_matrix = pickle.load(open("data/ratings_matrix.pkl", "rb"))
idx_to_mid = pickle.load(open("data/idx_to_mid.pkl", "rb"))
mid_to_idx = pickle.load(open("data/mid_to_idx.pkl", "rb"))
uid_to_idx = pickle.load(open("data/uid_to_idx.pkl", "rb"))
idx_to_uid = pickle.load(open("data/idx_to_uid.pkl", "rb"))

**Q2**. Because the dataset is slightly different from what we have been used to (X as features, y as target), the usual `train_test_split` method from scikit-learn does not apply.

Hopefully, `lightfm` comes with a `random_train_test_split` located into `cross_validation` dedicated to this usecase 🙂

Split the data randomly into a `train` matrix and a `test` matrix with 20% of interactions into the test set.

In [3]:
!pip install lightfm

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [23 lines of output]
  Compiling without OpenMP support.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-3.8\lightfm
  copying lightfm\data.py -> build\lib.win-amd64-3.8\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-3.8\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-3.8\lightfm
  copying lightfm\_lightfm_fast.py -> build\lib.win-amd64-3.8\lightfm
  copying lightfm\__init__.py -> build\lib.win-amd64-3.8\lightfm
  creating build\lib.win-amd64-3.8\lightfm\datasets
  copying lightfm\datasets\movielens.py -> build\lib.win-amd64-3.8\lightfm\datasets
  copying lightfm\datasets\stackexchange.py -> build\lib.win-amd64-3.8\lightfm\datasets
  copying lightfm\datasets\_common.py -> build\lib


  Using cached lightfm-1.16.tar.gz (310 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for lightfm
Failed to build lightfm
  Running setup.py install for lightfm: started
  Running setup.py install for lightfm: finished with status 'error'


In [4]:
conda install --channel https://conda.anaconda.org/mlgill lightfm

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - lightfm

Current channels:

  - https://conda.anaconda.org/mlgill/win-64
  - https://conda.anaconda.org/mlgill/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [7]:
conda install -c conda-forge lightfm

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Ben\anaconda3

  added / updated specs:
    - lightfm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.13.0               |   py38haa244fe_1        1016 KB  conda-forge
    lightfm-1.16               |   py38h347fdf6_2         357 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following NEW packages will be INSTALLED:

  lightfm            conda-forge/win-64::lightfm-1.16-py38h347fdf6_2

The following packages will be UPDATED:

  conda                               4.10.3-py38haa244fe_2 --> 4.13.0-py38haa244fe_1




lightfm-1.16         | 357 KB    |            |   0% 
lightfm-1.16         | 357 KB    | 4          |   4% 
lightfm-1.16         | 357 KB    | ########## | 100% 
lightfm-1

In [8]:
import numpy as np
from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(ratings_matrix, test_percentage=0.2, random_state= np.random.RandomState(0))

C:\Users\Ben\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


**Q3**. Train a LightFM model for 10 epochs. You can use the parameter `loss="warp"`.

In [9]:
from lightfm import LightFM

model = LightFM(no_components=100, loss="warp", random_state=0)

model.fit(train, epochs=10, verbose=True)

Epoch: 100%|██████████| 10/10 [00:20<00:00,  2.08s/it]


**Q4**. Evaluate your model on your test set. You can use the `precision_at_k` metric implemented in the LightFM library.

In [10]:
from lightfm.evaluation import precision_at_k

k = 5
precision_k = precision_at_k(model, test, train, k=k).mean()

print("Precision at k: ", k, "is", precision_k)

Precision at k:  5 is 0.29261085


**Q5**. What does the attribute `item_embeddings` of `model` contains?  This will be the heart of your recommendation engine! 💟 So make sure you understand fully what it contains.

In [11]:
print(model.item_embeddings.shape)

(3650, 100)


**Q6**. We just trained a model that factorized our ratings matrix into a U matrix of shape (n_users, no_components) : `model.user_embeddings` ; and V matrix of shape (n_movies, no_components) : `model.item_embeddings`).

Now we want to compute **similarity between each pair of movies**.

> 🔦 **Hint**: For the similarity distance we can either use `cosine_similarity` function or `pearson_similarity`:
> - **Cosine similarity** between two vectors, or matrices X and Y is given by:
> ``` python
> from sklearn.metrics.pairwise import cosine_similarity
> cosine_similarity(X, Y)
> ```
> - **Pearson similarity** between two vectors, or matrices X and Y is given by:
> ``` python
> import numpy as np
> np.corrcoef(X, Y)
> ```

Compute the `similarity_scores` of size (n_movies, n_movies), containing for each element (i, j) the similarity between movie of index i and movie of index j.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(model.item_embeddings)
print(similarity_scores.shape)
similarity_scores

(3650, 3650)


array([[ 0.9999996 ,  0.13317415,  0.24826698, ..., -0.42391142,
         0.04463575, -0.40535694],
       [ 0.13317415,  0.9999999 ,  0.1236582 , ..., -0.156093  ,
        -0.31249687,  0.03511212],
       [ 0.24826698,  0.1236582 ,  1.0000002 , ..., -0.28943077,
        -0.33148974,  0.12372942],
       ...,
       [-0.42391142, -0.156093  , -0.28943077, ...,  0.99999994,
         0.56635493,  0.13436124],
       [ 0.04463575, -0.31249687, -0.33148974, ...,  0.56635493,
         0.99999994, -0.2925488 ],
       [-0.40535694,  0.03511212,  0.12372942, ...,  0.13436124,
        -0.2925488 ,  0.99999994]], dtype=float32)

**Q7**. For movie of idx 20, what are the idx of the 10 most similar movies?

In [ ]:
idx = 20
similarity_idx = similarity_scores[idx]
ranked_idx = np.argsort(-similarity_idx)
ranked_mid = [idx_to_mid[x] for x in ranked_idx]
for mid in ranked_mid[:10]:
    print(movies[movies.movieId == mid]["title"])

**Q8**. Let's now test our engine! Suppose we have an user that likes **Toy Story** 🧸 (movie_id = 1). Which movies would you recommend to that user? In other words, which movies are the most similar to the movie Toy Story 

> ⚠️ **Warning**: Remember that your `similarity_scores` works with `idx` and you have the `movie_id` associated to your movie.

Retrieve the **top 5 recommendations**.

In [ ]:
idx = mid_to_idx[1]
similarity_idx = similarity_scores[idx]
ranked_idx = np.argsort(-similarity_idx)
ranked_mid = [idx_to_mid[x] for x in ranked_idx]
for mid in ranked_mid[:5]:
    print(movies[movies.movieId == mid]["title"])

As the next step is to **deploy your model**, you need now to: 

**Q9**. Save your `similarity_scores` into pickle format. Save also `movies` DataFrame into pickle format. Save them at the `data/netflix` directory at the root of the repository.

In [ ]:
directory = "./data"
pickle.dump(similarity_scores, open(directory + "./similarity_scores.pkl", "wb"))
pickle.dump(movies, open(directory + "./movies.pkl", "wb"))

**Q10**. Encapsulate the previous code into functions, especially you will need:
- `get_sim_scores(mid)` function that returns the vector of the similarity scores `sims` between a movie `mid` and all the other movies
- `get_ranked_recos(sims)` that returns for a vector of similarity scores `sims` the list of all ranked recommendations (n_movies) (from most recommended to least recommended) - in the format list of (mid, score, name) tuple.

In [ ]:
def movie_name(mid, movies):
    try:
        name = movies.loc[movies.movieId == mid].title.values[0]
    except:
        name = "Unkown"
    return name
        
def get_scores(mid):
    idx = mid_to_idx[mid]
    sims = similarity_scores[idx]
    return sims

def get_ranked(sims, movies):
    recos = []
    for idx in np.argsort(-sims):
        mid = idx_to_mid[idx]
        name = movie_name(mid, movies)
        score = sims[idx]
        recos.append((mid, score, name))
    return recos

def get_reccomendations(mid, movies, k):
    sim_scores = get_scores(mid)
    return get_ranked(sim_scores, movies)[:k]

In [ ]:
get_reccomendations(3, movies, 10)

If you have extra time, feel free now to improve your recommendation engine!